##### Grupo 2 - ADRIÁN TURIEL CHARRO, HONG XIANG CHEN, PABLO CIDONCHA CÓZAR
# Práctica 1. Resolución de problemas con búsqueda. Parte B.

In [1]:
cd ../Practica1/aima-python

C:\Users\aturi\Ejercicios Python\Practica1\aima-python


In [34]:
from search import *

In [35]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [36]:
class Puente(Problem):

    def init(self, initial=None, goal=(300, (), (10, 30, 60, 80, 120), 1) ):
        # Tiempo / Gente orilla A / Gente orilla B / Orilla literna
        self.initial = initial
        self.goal = goal


    def actions(self, state):
        tiempo, orillaA, orillaB, linterna = state
        accs=list()

        longX = len(orillaA)
        longY = len(orillaA)

        if linterna == 0: 
            for x in range(longX):
                for y in range(x+1, longY):
                    if(tiempo + max(orillaA[x], orillaA[y]))<=300:
                        accs.append(('Ir', (orillaA[x], orillaA[y])))
        else:
            for x in orillaB:
                accs.append(('Volver', (x)))
        return accs

    def result(self, state, action):
        tiempo, orillaA, orillaB, linterna = state
        mover, valores = action
        listA = list(orillaA)
        listB = list(orillaB)

        if mover=='Ir':
            x, y = valores
            listA.remove((x))
            listA.remove((y))
            listB.append((x))
            listB.append((y))
            linterna=1
            tiempo += max(x,y)
        elif mover=='Volver':
            listA.append((valores))
            listB.remove((valores))
            linterna=0
            tiempo += valores

        return tiempo, tuple(listA), tuple(listB), linterna

    def goal_test(self, state):
        tiempo, a, b, orilla = state
        return orilla==1 and len(b) == 5 and tiempo <= 300

    def coste_de_aplicar_accion(self, estado, accion):
        mover, valores = accion

        if mover == 'Ir':
            x,y = valores
            return max(x, y)
        else:
            return valores


In [37]:
def linear(node):
        tiempo, orillaA, orillaB, linterna = node.state
        restantesOrillaA = len(orillaA)
        if linterna == 0:
            return restantesOrillaA*2 - 1
        else:
            return restantesOrillaA*2

In [38]:
p = Puente((0, (10, 30, 60, 80, 120), (), 0))
p.initial

(0, (10, 30, 60, 80, 120), (), 0)

In [39]:
depth_first_graph_search(p).solution()

[('Ir', (10, 60)),
 ('Volver', 10),
 ('Ir', (30, 10)),
 ('Volver', 10),
 ('Ir', (80, 120)),
 ('Volver', 30),
 ('Ir', (10, 30))]

In [40]:
p.coste_de_aplicar_accion(p.initial, ('Ir', (10, 120)))

120

In [41]:
%%timeit
breadth_first_tree_search(p).solution()

8.49 ms ± 742 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%%timeit
breadth_first_graph_search(p).solution()

91.9 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%%timeit
depth_first_tree_search(p).solution()

5.37 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
depth_first_graph_search(p).solution()

7.06 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%%timeit
best_first_graph_search(p, linear).solution()

1.12 ms ± 63.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [46]:
%%timeit
astar_search(p, linear).solution()

192 ms ± 4.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit
uniform_cost_search(p).solution()

195 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Memoria
Hemos ejecutado los distintos algoritmos de búsqueda con el timeit para ver cuál es el más eficiente de todos ellos, y sacar nuestras propias conclusiones. En este caso al ser un ejemplo pequeño, podríamos usar cualquier algoritmo, ya que no hay grandes diferencias de tiempos entre ellos, sin embargo, dentro de todos ellos cabe destacar que hemos utilizado el algoritmo de primero el mejor para resolverlo ya que es con el que hemos obtenido resultado más rápidos, y si nos paramos a analizarlo tiene sentido porque expande las ramas del nodo más prometedor del nivel actual, y así sucesivamente hasta llegar a encontrar una solución óptima y eficiente.

Cuando estudiamos los nodos de un nivel, gestionamos los nodos abiertos con una cola de prioridad, donde estarán ordenados por la evaluación de su heurística. La nuestra estudia el número de viajes que necesitamos hacer para que todos crucen el puente, en función de la orilla en la que estemos y de las personas que quedan en la orilla A. Básicamente contamos el número de viajes, y se trata de un algoritmo de búsqueda linear. La heurística que hemos creado no es admisible, ya que el coste real es menor que el resultado que obtenemos, por lo tanto, tampoco es consistente. Explicación de por qué es mala nuestra primera heurística: la información que tenemos inicialmente en el primer nodo es: 3 personas en la orilla A y 0 en B; a raíz de eso tendremos 3 nodos en la siguiente altura (las 3 combinaciones posibles al viajar 2 personas), pero en todos esos casos, habrá 1 persona en A y 2 en B; y aplicando nuestra heurística, que busca el menor número de viajes, será ineficaz, ya que siempre va a dar la misma evaluación, pues al final acaba recorriendo el árbol como un algoritmo de profundidad. Sabemos que no es una buena heurística, pero la comprendemos y este es nuestro análisis.


## Complejidad
La complejidad del algoritmo primero el mejor es O(r^m) dónde r es el factor de ramificación, y m la profundidad máxima del árbol de búsqueda. Factor de ramificación es: 10 (Número de combinaciones totales por pares de elementos) m: 9 (según nuestra heurística).

O(10^9)

In [48]:
%%timeit
best_first_graph_search(p, linear).solution()

963 µs ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
